In [1]:
import pandas as pd
import numpy as np
import os
import torch
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DistilBertForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)
import evaluate

/Users/asherkhan/Coding Projects/Python/Natural Language Processing/705.641-nlp-final-project/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["TOKENIZERS_PARALLELISM"] = "true"
torch.set_float32_matmul_precision("high")

In [3]:
df = pd.read_csv('data/data.csv').sample(frac=0.05, random_state=42).reset_index(drop=True)
df = df.rename(columns={"source": "label"})
df["label"] = df["label"].map(lambda x: 1 if x == "Human" else 0)

In [4]:
df.shape

(39446, 5)

In [5]:
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [6]:
train_dataset = train_dataset.remove_columns(
    ['prompt_id', 'text_length', 'word_count', '__index_level_0__']
)

test_dataset = test_dataset.remove_columns(
    ['prompt_id', 'text_length', 'word_count', '__index_level_0__']
)

In [7]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length")

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset  = test_dataset.map(tokenize, batched=True)

Map: 100%|██████████| 3945/3945 [00:00<00:00, 4356.91 examples/s]


In [8]:
train_dataset = train_dataset.remove_columns(['text'])

test_dataset = test_dataset.remove_columns(['text'])

In [9]:
train_dataset.set_format(type=None)
test_dataset.set_format(type=None)

In [10]:
class SafeDistilBert(DistilBertForSequenceClassification):
    def forward(self, *args, num_items_in_batch=None, **kwargs):
        return super().forward(*args, **kwargs)


model = SafeDistilBert.from_pretrained(
    model_name,
    num_labels=2
)

model.to("mps")
model = torch.compile(model)

Some weights of SafeDistilBert were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=20,
    per_device_eval_batch_size=20,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    gradient_accumulation_steps=1,
    fp16=True,                         
    dataloader_num_workers=4,
    remove_unused_columns=False
)

metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)
    return metric.compute(predictions=preds, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


/var/folders/_g/_gxkj1q5733gjvsc9qys_70c0000gn/T/ipykernel_31125/1087674290.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/Users/asherkhan/Coding Projects/Python/Natural Language Processing/705.641-nlp-final-project/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
W1203 21:23:25.963000 31125 torch/_inductor/utils.py:1613] [0/0] Not enough SMs to use max_autotune_gemm mode


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
result = trainer.evaluate()
print(result)